# Timescale Vector (Postgres) self-querying | Timescale Vector（Postgres）自己問い合わせ[Timescale Vector](https://www.timescale.com/ai)はAIアプリケーション向けのPostgreSQL++です。これにより、`PostgreSQL`で数十億のベクトル埋め込みを効率的に保存およびクエリすることが可能です。> [Timescale Vector](https://www.timescale.com/ai) is PostgreSQL++ for AI applications. It enables you to efficiently store and query billions of vector embeddings in `PostgreSQL`.このノートブックでは、Postgresベクターデータベース（`TimescaleVector`）を使用してセルフクエリを実行する方法を示しています。ノートブックでは、TimescaleVectorベクターストアをラップした`SelfQueryRetriever`のデモを行います。> This notebook shows how to use the Postgres vector database (`TimescaleVector`) to perform self-querying. In the notebook we'll demo the `SelfQueryRetriever` wrapped around a TimescaleVector vector store.## What is Timescale Vector? | Timescale Vectorとは何ですか？**[Timescale Vector](https://www.timescale.com/ai)は、AIアプリケーション向けのPostgreSQL++です。**> **[Timescale Vector](https://www.timescale.com/ai) is PostgreSQL++ for AI applications.**`PostgreSQL`で数百万のベクトル埋め込みを効率的に保存し、クエリすることができるTimescale Vectorを紹介します。> Timescale Vector enables you to efficiently store and query millions of vector embeddings in `PostgreSQL`.* `pgvector`をDiskANNにインスパイアされたインデックス作成アルゴリズムを通じて、10億以上のベクトルに対するより高速で正確な類似性検索機能を強化します。  > Enhances `pgvector` with faster and more accurate similarity search on 1B+ vectors via DiskANN inspired indexing algorithm.* 自動的な時間ベースのパーティショニングとインデックス作成により、高速な時間ベースのベクトル検索を可能にします。  > Enables fast time-based vector search via automatic time-based partitioning and indexing.* ベクトルの埋め込みと関連データを問い合わせるための、馴染みのあるSQLインターフェースを提供します。  > Provides a familiar SQL interface for querying vector embeddings and relational data.Timescale Vectorは、POCから本番環境まであなたと共にスケールするAI用クラウドPostgreSQLです。> Timescale Vector is cloud PostgreSQL for AI that scales with you from POC to production:* リレーショナルメタデータ、ベクトル埋め込み、時系列データを一つのデータベースに保存することで、操作を簡素化します。  > Simplifies operations by enabling you to store relational metadata, vector embeddings, and time-series data in a single database.* 堅牢なPostgreSQLの基盤の恩恵を享受し、ストリーミングバックアップやレプリケーション、高可用性、行レベルのセキュリティといったエンタープライズグレードの機能を備えています。  > Benefits from rock-solid PostgreSQL foundation with enterprise-grade feature liked streaming backups and replication, high-availability and row-level security.* エンタープライズグレードのセキュリティとコンプライアンスにより、安心してご利用いただける体験を実現します。  > Enables a worry-free experience with enterprise-grade security and compliance.## How to access Timescale Vector | Timescale Vectorにアクセスする方法Timescale Vectorは、クラウドPostgreSQLプラットフォームである[Timescale](https://www.timescale.com/ai)で利用可能です。（現時点では自己ホスト型のバージョンはありません。）> Timescale Vector is available on [Timescale](https://www.timescale.com/ai), the cloud PostgreSQL platform. (There is no self-hosted version at this time.)LangChainのユーザーは、Timescale Vectorの90日間の無料トライアルを利用できます。> LangChain users get a 90-day free trial for Timescale Vector.* 始めるには、Timescaleに[サインアップ](https://console.cloud.timescale.com/signup?utm_campaign=vectorlaunch\&utm_source=langchain\&utm_medium=referral)して、新しいデータベースを作成し、このノートブックに従って進めてください！  > To get started, [signup](https://console.cloud.timescale.com/signup?utm_campaign=vectorlaunch\&utm_source=langchain\&utm_medium=referral) to Timescale, create a new database and follow this notebook!* 詳細とパフォーマンスベンチマークについては、[Timescale Vectorの解説ブログ](https://www.timescale.com/blog/how-we-made-postgresql-the-best-vector-database/?utm_campaign=vectorlaunch\&utm_source=langchain\&utm_medium=referral)をご覧ください。  > See the [Timescale Vector explainer blog](https://www.timescale.com/blog/how-we-made-postgresql-the-best-vector-database/?utm_campaign=vectorlaunch\&utm_source=langchain\&utm_medium=referral) for more details and performance benchmarks.* PythonでTimescale Vectorを使用する詳細については、[インストール手順](https://github.com/timescale/python-vector)をご覧ください。  > See the [installation instructions](https://github.com/timescale/python-vector) for more details on using Timescale Vector in python.

## Creating a TimescaleVector vectorstore | TimescaleVectorベクトルストアの作成まず、Timescale Vector vectorstoreを作成し、それにいくつかのデータを投入します。映画の要約を含む小さなデモ文書セットを用意しました。> First we'll want to create a Timescale Vector vectorstore and seed it with some data. We've created a small demo set of documents that contain summaries of movies.注意：self-query retrieverを使用するには、`lark`がインストールされている必要があります（`pip install lark`でインストールできます）。また、`timescale-vector`パッケージも必要です。> NOTE: The self-query retriever requires you to have `lark` installed (`pip install lark`). We also need the `timescale-vector` package.

In [1]:
#!pip install lark


In [2]:
#!pip install timescale-vector


この例では、`OpenAIEmbeddings`を使用するので、あなたのOpenAI APIキーを読み込みましょう。> In this example, we'll use `OpenAIEmbeddings`, so let's load your OpenAI API key.

In [1]:
# Get openAI api key by reading local .env file
# The .env file should contain a line starting with `OPENAI_API_KEY=sk-`
import os

from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
# Alternatively, use getpass to enter the key in a prompt
# import os
# import getpass
# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

PostgreSQLデータベースに接続するには、新しいデータベースを作成した後にダウンロードしたチートシートまたは`.env`ファイルで見つけることができるサービスURIが必要です。> To connect to your PostgreSQL database, you'll need your service URI, which can be found in the cheatsheet or `.env` file you downloaded after creating a new database.まだの方は、[Timescaleにサインアップ](https://console.cloud.timescale.com/signup?utm_campaign=vectorlaunch\&utm_source=langchain\&utm_medium=referral)して、新しいデータベースを作成してください。> If you haven't already, [signup for Timescale](https://console.cloud.timescale.com/signup?utm_campaign=vectorlaunch\&utm_source=langchain\&utm_medium=referral), and create a new database.URIは次のようになります：`postgres://tsdbadmin:<password>@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require`> The URI will look something like this: `postgres://tsdbadmin:<password>@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require`

In [2]:
# Get the service url by reading local .env file
# The .env file should contain a line starting with `TIMESCALE_SERVICE_URL=postgresql://`
_ = load_dotenv(find_dotenv())
TIMESCALE_SERVICE_URL = os.environ["TIMESCALE_SERVICE_URL"]

# Alternatively, use getpass to enter the key in a prompt
# import os
# import getpass
# TIMESCALE_SERVICE_URL = getpass.getpass("Timescale Service URL:")

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores.timescalevector import TimescaleVector

embeddings = OpenAIEmbeddings()

こちらは、このデモで使用するサンプルドキュメントです。データは映画に関するもので、特定の映画についての情報が含まれたコンテンツとメタデータのフィールドがあります。> Here's the sample documents we'll use for this demo. The data is about movies, and has both content and metadata fields with information about particular movie.

In [4]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "science fiction",
            "rating": 9.9,
        },
    ),
]

最後に、Timescale Vector vectorstoreを作成しましょう。コレクション名は、ドキュメントが保存されるPostgreSQLテーブルの名前になりますので、この点に注意してください。> Finally, we'll create our Timescale Vector vectorstore. Note that the collection name will be the name of the PostgreSQL table in which the documents are stored in.

In [5]:
COLLECTION_NAME = "langchain_self_query_demo"
vectorstore = TimescaleVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    service_url=TIMESCALE_SERVICE_URL,
)

## Creating our self-querying retriever | 自己問い合わせ型リトリーバーの作成これで、私たちのリトリーバーをインスタンス化することができます。これを行うためには、ドキュメントがサポートしているメタデータフィールドに関する情報と、ドキュメント内容の簡潔な説明を事前に提供する必要があります。> Now we can instantiate our retriever. To do this we'll need to provide some information upfront about the metadata fields that our documents support and a short description of the document contents.

In [14]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever

# Give LLM info about the metadata fields
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"

# Instantiate the self-query retriever from an LLM
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

## Self Querying Retrieval with Timescale Vector | タイムスケールベクトルを用いた自己問い合わせ検索さあ、私たちのリトリーバーを実際に使ってみましょう！> And now we can try actually using our retriever!以下のクエリを実行し、自然言語でクエリ、フィルター、複合フィルター（ANDやORを使ったフィルター）をどのように指定できるかに注目してください。そして、self-query retrieverがそのクエリをSQLに変換し、Timescale Vector（Postgres）のvectorstoreで検索を実施する様子を観察してください。> Run the queries below and note how you can specify a query, filter, composite filter (filters with AND, OR) in natural language and the self-query retriever will translate that query into SQL and perform the search on the Timescale Vector (Postgres) vectorstore.これは、セルフクエリリトリーバーの力を示しています。あなたやユーザーが直接SQLを書かなくても、ベクターストア上で複雑な検索を実行することができます！> This illustrates the power of the self-query retriever. You can use it to perform complex searches over your vectorstore without you or your users having to write any SQL directly!

In [15]:
# This example only specifies a relevant query
retriever.get_relevant_documents("What are some movies about dinosaurs")

/Users/avtharsewrathan/sideprojects2023/timescaleai/tsv-langchain/langchain/libs/langchain/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='dinosaur' filter=None limit=None


[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'year': 1993, 'genre': 'science fiction', 'rating': 7.7}),
 Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'year': 1993, 'genre': 'science fiction', 'rating': 7.7}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'year': 1995, 'genre': 'animated'}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'year': 1995, 'genre': 'animated'})]

In [16]:
# This example only specifies a filter
retriever.get_relevant_documents("I want to watch a movie rated higher than 8.5")

query=' ' filter=Comparison(comparator=<Comparator.GT: 'gt'>, attribute='rating', value=8.5) limit=None


[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'year': 1979, 'genre': 'science fiction', 'rating': 9.9, 'director': 'Andrei Tarkovsky'}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'year': 1979, 'genre': 'science fiction', 'rating': 9.9, 'director': 'Andrei Tarkovsky'}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'year': 2006, 'rating': 8.6, 'director': 'Satoshi Kon'}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'year': 2006, 'rating': 8.6, 'director': 'Satoshi Kon'})]

In [17]:
# This example specifies a query and a filter
retriever.get_relevant_documents("Has Greta Gerwig directed any movies about women")

query='women' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Greta Gerwig') limit=None


[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'year': 2019, 'rating': 8.3, 'director': 'Greta Gerwig'}),
 Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'year': 2019, 'rating': 8.3, 'director': 'Greta Gerwig'})]

In [18]:
# This example specifies a composite filter
retriever.get_relevant_documents(
    "What's a highly rated (above 8.5) science fiction film?"
)

query=' ' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='rating', value=8.5), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='science fiction')]) limit=None


[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'year': 1979, 'genre': 'science fiction', 'rating': 9.9, 'director': 'Andrei Tarkovsky'}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'year': 1979, 'genre': 'science fiction', 'rating': 9.9, 'director': 'Andrei Tarkovsky'})]

In [11]:
# This example specifies a query and composite filter
retriever.get_relevant_documents(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

query='toys' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GT: 'gt'>, attribute='year', value=1990), Comparison(comparator=<Comparator.LT: 'lt'>, attribute='year', value=2005), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='animated')]) limit=None


[Document(page_content='Toys come alive and have a blast doing so', metadata={'year': 1995, 'genre': 'animated'})]

### Filter k | フィルター kself query retrieverを使用して、取得するドキュメントの数`k`を指定することもできます。> We can also use the self query retriever to specify `k`: the number of documents to fetch.これを行うには、コンストラクタに `enable_limit=True` を渡すことで実現できます。> We can do this by passing `enable_limit=True` to the constructor.

In [19]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    verbose=True,
)

In [22]:
# This example specifies a query with a LIMIT value
retriever.get_relevant_documents("what are two movies about dinosaurs")

query='dinosaur' filter=None limit=2


[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'year': 1993, 'genre': 'science fiction', 'rating': 7.7}),
 Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'year': 1993, 'genre': 'science fiction', 'rating': 7.7})]